In [2]:
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

## Read all input files and merge into one output

In [21]:
path = os.getcwd()+"\\output"
files = os.listdir(path)

files_xls = [r"C:\Users\Titas\Desktop\project\11b\jupyter notebook\output\\"+f for f in files if f[-4:] == 'xlsx']
files_xls

df = pd.DataFrame()
for f in files_xls:
    data = pd.read_excel(f, 'Sheet1')
    df = df.append(data)
df.to_excel("output.xlsx", index= False)

In [78]:
df = pd.DataFrame()
df = pd.read_excel("output.xlsx", index= False)
print(len(df))
df.head()

1313


,Adresas,Aukštas:,Aukštų sk.:,Buto numeris:,Kaina mėn.:,Kambarių sk.:,Metai:,Namo numeris:,Nuoroda,Pastato tipas:,Peržiūrėjo,Plotas:,Redaguotas,Įdėtas,Įrengimas:,Šildymas:
0,https://www.aruodas.lt/butai-vilniuje-pasilaic...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.aruodas.lt/butu-nuoma-vilniuje-pas...,3.0,9.0,NaN,450 €,2.0,2007,NaN,NaN,Mūrinis,NaN,64 m²,NaN,NaN,Įrengtas ...,Centrinis kolektorinis
2,https://www.aruodas.lt/butu-nuoma-vilniuje-zve...,4.0,4.0,NaN,699 €,2.0,2019,10,NaN,Mūrinis,NaN,"41,47 m²",NaN,NaN,NaN,"Centrinis, elektra"
3,https://www.aruodas.lt/butu-nuoma-vilniuje-sen...,3.0,3.0,NaN,800 €,3.0,1940,NaN,NaN,Mūrinis,NaN,71 m²,NaN,NaN,Įrengtas ...,Dujinis
4,https://www.aruodas.lt/butu-nuoma-vilniuje-zve...,2.0,3.0,NaN,400 €,2.0,1969,NaN,NaN,Mūrinis,NaN,"41,84 m²",NaN,NaN,Įrengtas ...,Centrinis


## Clean data

In [79]:
# Delete rows with Nan values in Kaina men
df =df.drop([df[pd.isna(df["Kaina mėn.:"])].index][0])
print(len(df))

1266


In [80]:
# replace characters
df['Kaina mėn.:'] = df['Kaina mėn.:'].str.replace('€','')
df['Kaina mėn.:'] = df['Kaina mėn.:'].str.replace(' ','')
df['Plotas:'] = df['Plotas:'].str.replace('m²','')
df['Plotas:'] = df['Plotas:'].str.replace(',','.')

df['Metai:'] = df["Metai:"].astype(str)
df[['Metai','Renovacija']] = df['Metai:'].str.split(',',expand=True)
# clean year data
df['Metai'] = df['Metai'].str[:4]
df['Renovacija'] = df['Renovacija'].str[:5]
# replace None with 0 in Renovacija
df['Renovacija'] = df['Renovacija'].fillna(0)
df.head()

df.to_excel("outputAfterCleaning.xlsx", index= False)

In [90]:
df.loc[df[df['Adresas'].str.contains('pasilaiciuose').values].index,"Rajonas"] = "pasilaiciuose"
df.loc[df[df['Adresas'].str.contains('zveryne').values].index,"Rajonas"] = "zveryne"
df.loc[df[df['Adresas'].str.contains('pilaiteje').values].index,"Rajonas"] = "pilaiteje"
df.loc[df[df['Adresas'].str.contains('senamiestyje').values].index,"Rajonas"] = "senamiestyje"
df.loc[df[df['Adresas'].str.contains('naujamiestyje').values].index,"Rajonas"] = "naujamiestyje"

df.loc[df[df['Adresas'].str.contains('uzupyje').values].index,"Rajonas"] = "uzupyje"
df.loc[df[df['Adresas'].str.contains('snipiskese').values].index,"Rajonas"] = "snipiskese"
df.loc[df[df['Adresas'].str.contains('antakalnyje').values].index,"Rajonas"] = "antakalnyje"
df.loc[df[df['Adresas'].str.contains('zirmunuose').values].index,"Rajonas"] = "zirmunuose"

df['Rajonas'] = df['Rajonas'].fillna("None")
RajonoKodavimas = LabelEncoder()
df['RajonoKodavimas'] = RajonoKodavimas.fit_transform(df['Rajonas'])

# df.to_excel("outputAfterCleaningdPas.xlsx", index= False)

In [84]:
# encode the building type
df =df.drop([df[pd.isna(df["Pastato tipas:"])].index][0])
PastatoTipas = LabelEncoder()

df['Pastato tipas'] = PastatoTipas.fit_transform(df['Pastato tipas:'])

In [85]:
# delete repeating columns
del df['Metai:']
del df['Pastato tipas:']

In [86]:
# save df with features and target columns
df =df.drop([df[pd.isna(df["Aukštas:"])].index][0])
df =df.drop([df[pd.isna(df["Kambarių sk.:"])].index][0])

df_train = df[['Metai', 'Plotas:', 'Kambarių sk.:', 'Aukštas:', 'Pastato tipas','Renovacija','Kaina mėn.:', 'RajonoKodavimas']]

In [91]:
# check for null values
total = df_train.isna().sum().sort_values(ascending=False)
total

RajonoKodavimas    0
Kaina mėn.:        0
Renovacija         0
Pastato tipas      0
Aukštas:           0
Kambarių sk.:      0
Plotas:            0
Metai              0
dtype: int64

In [92]:
# split by features and target to X and y
X = df[['Metai', 'Plotas:', 'Kambarių sk.:', 'Aukštas:', 'Pastato tipas','Renovacija','RajonoKodavimas']]
y = df['Kaina mėn.:']

# split into train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0)
print(X_train.shape, y_train.shape)

# train the model
model = RandomForestRegressor(random_state=0, max_depth=5)
model.fit(X_train, y_train)

# calculate the mean_absolute_error for train and test data
prediction_train = model.predict(X_train)
train = mean_absolute_error(prediction_train, y_train)

prediction_test = model.predict(X_test)
test = mean_absolute_error(prediction_test, y_test)

print("train:", train,"test:", test)

(947, 7) (947,)
train: 119.20666638205884 test: 331.819644337792


C:\Users\Titas\Anaconda3\envs\b11\lib\site-packages\sklearn\ensemble\forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [93]:
model.feature_importances_

array([0.08581401, 0.56636701, 0.27538598, 0.01041491, 0.00212124,
       0.        , 0.05989685])

In [ ]:
from sklearn.externals import joblib
joblib.dump(model, 'my_model.pkl', compress=9)

# model_clone = joblib.load('my_model.pkl')

In [ ]:
model_clone = joblib.load('my_model.pkl')
flat = [[2008,46,2,1,0,0]]
print(model.predict(flat))

In [104]:
df['Rajonas'].value_counts()

None             366
senamiestyje     280
naujamiestyje    161
snipiskese        86
antakalnyje       78
pasilaiciuose     73
zveryne           68
pilaiteje         53
zirmunuose        53
uzupyje           45
Name: Rajonas, dtype: int64